### List of all products 

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'



url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
for i in range(0, 25):
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)
'''

Page 0 loaded successfully.
Page 1 loaded successfully.
Page 2 loaded successfully.
Page 3 loaded successfully.
Page 4 loaded successfully.
Page 5 loaded successfully.
Page 6 loaded successfully.
Page 7 loaded successfully.
Page 8 loaded successfully.
Page 9 loaded successfully.
Page 10 loaded successfully.
Page 11 loaded successfully.
Page 12 loaded successfully.
Page 13 loaded successfully.
Page 14 loaded successfully.
Page 15 loaded successfully.
Page 16 loaded successfully.
Page 17 loaded successfully.
Page 18 loaded successfully.
Page 19 loaded successfully.
Page 20 loaded successfully.
Page 21 loaded successfully.
Page 22 loaded successfully.
Page 23 loaded successfully.
Page 24 loaded successfully.


'\nurl = BASE_URL + ENDPOINT\n\nquery = {\n  \'page[number]\': 1,\n  \'page[size]\': 200\n}\n\nheaders = {\n  \'Authorization\': \'Token token=\' + KEY,\n  \'Accept\': \'application/json\',\n  \'Content-Type\': \'application/vnd.api+json\',\n  \'Accept-Encoding\': \'gzip,deflate\'\n}\n\nr = requests.get(url, params=query, headers=headers)\nprint("Status: " + str(r.status_code))\nif r.status_code == 200:\n    print(\'Page loaded successfully.\')\n    print(\'Generated in \' + str(r.json().get(\'meta\', {}).get(\'generated_in\', -1)) + \' milliseconds.\')\n    print(\'Next page\'s URL is: \' + r.json()[\'links\'].get(\'next\', \'UNKNOWN\'))\n    print(\'Barcodes on this page:\')\n    for product in r.json()[\'data\']:\n        productlist = []\n        for x in r.json()[\'data\']:\n            productlist.append(x[\'id\'])\nprint(productlist)\n'

### Getting list of useful characteristics from id

In [3]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations']['fr']
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations']['fr']
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
            
            dInfo[data['barcode']] = item    
            dIngredients[data['barcode']] = item_ing
            dNutrients[data['barcode']] = item_nut
        except:
            pass

CPU times: user 3min 57s, sys: 7.45 s, total: 4min 4s
Wall time: 19min 20s


### Translate ingredients

In [4]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
remove = []
for barcode, tmp in dIngredients.items():
    try:        
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_       
        name_ = ' '.join(tmp['name'][1].splitlines())
        language_json = language_translator.identify(name_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dInfo[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dIngredients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
            dNutrients[barcode]['name'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dInfo[barcode]['name'] = "en_kept, " + name_
            dIngredients[barcode]['name'] = "en_kept, " + name_
            dNutrients[barcode]['name'] = "en_kept, " + name_
                        
    except:
        remove.append(barcode)
        
    count += 1
    if count % 50 == 0:
        print("%s/%s items done!"%(count, len(dIngredients)))

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

50/4224 items done!
100/4224 items done!
150/4224 items done!
200/4224 items done!
250/4224 items done!
300/4224 items done!
350/4224 items done!
400/4224 items done!
450/4224 items done!
500/4224 items done!
550/4224 items done!
600/4224 items done!
650/4224 items done!
700/4224 items done!
750/4224 items done!
800/4224 items done!
850/4224 items done!
900/4224 items done!
950/4224 items done!
1000/4224 items done!
1050/4224 items done!
1100/4224 items done!
1150/4224 items done!
1200/4224 items done!
1250/4224 items done!
1300/4224 items done!
1350/4224 items done!
1400/4224 items done!
1450/4224 items done!
1500/4224 items done!
1550/4224 items done!
1600/4224 items done!
1650/4224 items done!
1700/4224 items done!
1750/4224 items done!
1800/4224 items done!
1850/4224 items done!
1900/4224 items done!
1950/4224 items done!
2000/4224 items done!
2050/4224 items done!
2100/4224 items done!
2150/4224 items done!
2200/4224 items done!
2250/4224 items done!
2300/4224 items done!
2350/422

In [5]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)

### Calculate climate impact!

In [3]:
import json

with open("dInfo.json", "r") as f_:
    dInfo = json.load(f_)
with open("dIngredients.json", "r") as f_:
    dIngredients = json.load(f_)
with open("dNutrients.json", "r") as f_:
    dNutrients = json.load(f_)

In [4]:
from food_db import build_database
ingredients_to_impact = build_database()

In [5]:
import re

ingredient_dict = {}

for item in list(dIngredients.items()):

    lItems = item[1]['ingredients'].split(',')[1:]

    quantity_dict = {}

    i = 1

    estimate = 50

    for l in lItems:

        percentage = re.findall(r'\d+.\d+%', l)

        if(len(percentage) == 0):

            quantity_dict[l] = estimate

        else:

            quantity_dict[l] = percentage[0].rstrip('%')

        i = i + 1

        estimate = estimate/2

    ingredient_dict[item[0]] = quantity_dict

In [10]:
import difflib
footprint_dict = {}

for item in final_dict.items():

    footprint = 0

    ingredients = item[1]['ingredients']

    quantity = float(final_dict[item[0]]['quantity'])

    for ing_key, ing_quantity in ingredients.items():

        print(ing_key)

        scores = []

        keys = []

        for key, value in ingredients_to_impact.items():

            keys.append(key)

            scores.append(difflib.SequenceMatcher(None, ing_key,key).ratio())

        matched_score = ingredients_to_impact[keys[scores.index(max(scores))]]

        print(quantity,ing_quantity,matched_score)

        footprint += float(quantity)*float(ing_quantity)*0.01*float(matched_score)

    footprint_dict[item[0]] = footprint

 r
(0.0, 50, 0.25)
 "Is"
(100.0, 50, 0.57)
 "Is"
(1000.0, 50, 0.57)
 r
(0.0, 50, 0.25)
 r
(0.0, 50, 0.25)
 "And"
(0.0, 50, 1.89)
 "H"
(0.0, 50, 0.44)
 "The"
(250.0, 50, 0.3)
 r
(0.0, 50, 0.25)
 "H"
(35.0, 50, 0.44)
 "The"
(750.0, 50, 0.3)
 "M"
(200.0, 50, 0.44)
 "And"
(0.0, 50, 1.89)
 r
(63.0, 50, 0.25)
 "And"
(250.0, 50, 1.89)
 "And"
(500.0, 50, 1.89)
 â
(0.0, 50, 0.44)
 r
(0.0, 50, 0.25)
 "H"
(50.0, 50, 0.44)
 "And"
(0.0, 50, 1.89)
 "And"
(500.0, 50, 1.89)
 â
(0.0, 50, 0.44)
 r
(190.0, 50, 0.25)
 "And"
(750.0, 50, 1.89)
 "The"
(1080.0, 50, 0.3)
 "H"
(0.0, 50, 0.44)
 c
(350.0, 50, 5.25)
 "Is"
(0.0, 50, 0.57)
 R
(500.0, 50, 0.44)
 "H"
(50.0, 50, 0.44)
 "The"
(400.0, 50, 0.3)
 "The"
(0.0, 50, 0.3)
 â
(0.0, 50, 0.44)
 "The"
(0.0, 50, 0.3)
 "The"
(180.0, 50, 0.3)
 â
(0.0, 50, 0.44)
 "The"
(180.0, 50, 0.3)
 "The"
(500.0, 50, 0.3)
 "The"
(180.0, 50, 0.3)
 "H"
(500.0, 50, 0.44)
 "The"
(500.0, 50, 0.3)
 "The"
(350.0, 50, 0.3)
 "And"
(100.0, 50, 1.89)
 "The"
(0.0, 50, 0.3)
 "And"
(500.0, 50, 1